# Plots and fit of all the detected clumps

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import warnings
warnings.filterwarnings('ignore')

%matplotlib notebook
import matplotlib.pylab as plt
from matplotlib.patches import Ellipse

import seaborn as sns
sns.set(style="whitegrid")

import numpy as np

from astropy.io import fits
from astropy import units as u
from astropy.stats import sigma_clipped_stats
from astropy.table import Table,Column

from astropy.cosmology import WMAP9 as cosmo

## Prepare tables for paper

1 ) Table with 'basic' properties:
    coordinates
    flux (I times channel width) Jy km/s
    Area (as measured, no correction)
    line width (as measured, no correction)

    
2 ) Table with corrected properties:
    
    mu
    L' CO
    L CO
    radius (with the correction)
    line width (with correction)
    
    Molecular Mass
    Virial Mass
    virial parameter
    
    Stellar Mass
    metallicity
    star formation density
    

First simpler table

In [2]:
def prepare_one_latex_entry_1(t,prefix):

    # Number of the clump
    output = '%s%d '%(prefix,t['clump_nb'])

    # coordinates in reasonable 
    output = output + '& %0.5f & %0.5f '%(t['x_cen'],t['y_cen'])

    # flux with upper and lower limits and in mJy km/s
    uperr  = np.max((t['flux_uerr']*1000,0.001))
    lowerr = np.max((t['flux_lerr']*1000,0.001))
    output = output + '& %0.3f$^{+%0.3f}_{-%0.3f}$ '%(t['flux']*1000,uperr,lowerr)

    # Area in arsec^2
    uperr  = np.max((t['area_exact_uerr']*(3600**2),0.001))
    lowerr = np.max((t['area_exact_lerr']*(3600**2),0.001))
    output = output + '& %0.3f$^{+%0.3f}_{-%0.3f}$ '%(t['area_exact']*(3600**2),uperr,lowerr)

    # Line width
    output = output + '& %0.1f$\pm$%0.1f '%(t['measured_line_width'],t['line_width_unc'])

    # Add end of line
    output = output + '\\\\'

    print(output)
    return output
    
    
def prepare_latex_table(table,prefix):
    for t in table:
        prepare_one_latex_entry_1(t,prefix)

Load data

In [3]:
as1063_tab1 = Table.read('../FindClumps/as1063_catalogue.fits')
as1063_tab1['clump_nb'] = range(len(as1063_tab1))

a521_tab1 = Table.read('../FindClumps/a521_catalogue.fits')
a521_tab1['clump_nb'] = range(len(a521_tab1))

snake_tab1 = Table.read('../FindClumps/snake_catalogue.fits')
snake_tab1['clump_nb'] = range(len(snake_tab1))

Table

In [4]:
prepare_latex_table(as1063_tab1,'AS-') 

head_a521 = a521_tab1[np.where(a521_tab1['mlt_region']==1)]
prepare_latex_table(head_a521,'A-H-') 
tailN_a521 = a521_tab1[np.where(a521_tab1['mlt_region']==2)]
prepare_latex_table(tailN_a521,'A-E-') 
tailS_a521 = a521_tab1[np.where(a521_tab1['mlt_region']==3)]
prepare_latex_table(tailS_a521,'A-W-') 

north_snake = snake_tab1[np.where(snake_tab1['mlt_region']==1)]
prepare_latex_table(north_snake,'M-N-') 
centre_snake = snake_tab1[np.where(snake_tab1['mlt_region']==2)]
prepare_latex_table(centre_snake,'M-C-') 
south_snake = snake_tab1[np.where(snake_tab1['mlt_region']==3)]
prepare_latex_table(south_snake,'M-S-') 

AS-0 & 342.17368 & -44.53189 & 1.974$^{+0.589}_{-0.740}$ & 0.032$^{+0.009}_{-0.010}$ & 16.1$\pm$0.8 \\
AS-1 & 342.17399 & -44.53236 & 15.319$^{+3.585}_{-3.145}$ & 0.130$^{+0.025}_{-0.024}$ & 83.3$\pm$2.7 \\
AS-2 & 342.17401 & -44.53255 & 2.192$^{+0.001}_{-0.410}$ & 0.039$^{+0.001}_{-0.009}$ & 42.9$\pm$0.4 \\
AS-3 & 342.17409 & -44.53249 & 4.603$^{+1.384}_{-1.500}$ & 0.073$^{+0.020}_{-0.021}$ & 39.2$\pm$0.5 \\
AS-4 & 342.17410 & -44.53262 & 6.460$^{+0.001}_{-0.583}$ & 0.083$^{+0.001}_{-0.005}$ & 23.6$\pm$0.1 \\
AS-5 & 342.17442 & -44.53292 & 0.978$^{+0.257}_{-0.978}$ & 0.018$^{+0.005}_{-0.018}$ & 16.9$\pm$9.8 \\
AS-6 & 342.17420 & -44.53271 & 3.130$^{+0.001}_{-0.538}$ & 0.049$^{+0.001}_{-0.008}$ & 23.6$\pm$0.1 \\
A-H-4 & 73.52863 & -10.22169 & 3.737$^{+0.001}_{-0.001}$ & 0.085$^{+0.001}_{-0.001}$ & 22.5$\pm$0.0 \\
A-H-8 & 73.52898 & -10.22171 & 27.110$^{+2.860}_{-1.621}$ & 0.188$^{+0.023}_{-0.012}$ & 47.5$\pm$0.2 \\
A-H-9 & 73.52877 & -10.22165 & 3.956$^{+0.001}_{-0.001}$ & 0.085$^{+0.0

Table 2

In [13]:
def prepare_one_latex_entry_2(t,prefix):
    
    # Number of the clump
    if t['radius_not_corrected'] == 1: # it is an upper limit
        output = '$\dagger$%s%d '%(prefix,t['clump_nb'])
    else:
        output = '%s%d '%(prefix,t['clump_nb'])
    
    # magnification
    output = output + '& %0.2f$\pm$%0.2f '%(t['mu'],t['mu_unc'])
    
    # LCO
    #output = output + '& %d$^{+%d}_{-%d}$ '%(t['L_CO43']*0.001,t['L_CO43_uerr']*0.001,t['L_CO43_lerr']*0.001)
    
    # L'CO
    output = output + '& %d$^{+%d}_{-%d}$ '%(t['Ll_CO43']*0.001,t['Ll_CO43_uerr']*0.001,t['Ll_CO43_lerr']*0.0001)
    
    # radius (corrected) in parsec
    if t['radius_not_corrected'] == 1: # it is an upper limit
        output = output + '& %d$_{-%d}$ '%(t['R'],t['R_lerr'])
    else:
        output = output + '& %d$^{+%d}_{-%d}$ '%(t['R'],t['R_uerr'],t['R_lerr'])
        
    # line width (corrected)
    output = output + '& %0.1f$\pm$%0.1f '%(t['line_width'],np.max((t['line_width_unc'],0.1)))

    # Molecular Masss
    output = output + '& %0.1f$^{+%0.1f}_{-%0.1f}$ '%(np.log10(t['Mass_mol']),t['Mass_mol_uerr']/t['Mass_mol'],t['Mass_mol_lerr']/t['Mass_mol'])
    
    # Stellar Masss
    #output = output + '& %0.1f$^{+%0.1f}_{-%0.1f}$ '%(np.log10(t['Mass_stellar']),t['Mass_stellar_uerr']/t['Mass_stellar'],t['Mass_stellar_lerr']/t['Mass_stellar'])

    # Virial Mass
    output = output + '& %0.1f$^{+%0.1f}_{-%0.1f}$ '%(np.log10(t['Mass_vir']),t['Mass_vir_uerr']/t['Mass_vir'],t['Mass_vir_lerr']/t['Mass_vir'])

    # virial parameter
    output = output + '& %0.1f '%(t['vir_param'])
     
    # Presssure
    output = output + '& %d$^{+%d}_{-%d}$ '%(t['P_int']*0.001,t['P_int_uerr']*0.001,t['P_int_lerr']*0.0001)
    
    #Gas density
    output = output + '& %d$^{+%d}_{-%d}$ '%(t['gas_density'],t['gas_density_uerr'],t['gas_density_lerr'])
    
    #Galactic distance
    #output = output + '& %d '%(t['Galactic_R']*1000)
    
    # star formation 
    #try:
        #output = output + '& %0.2f$\pm$%0.2f '%(t['SFR'],t['SFR_unc'])
    #except:
        #output = output + '& '

    # metallicity
    #try:
        #output = output + '& %0.2f$\pm$%0.2f '%(t['met'],t['met_unc'])
    #except:
        #output = output + '& '

    #end
    output = output + '\\\\'
    
    print(output)
    return output
    
def prepare_latex_table2(table,prefix):
    for t in table:
        prepare_one_latex_entry_2(t,prefix)

Load data

In [9]:
as1063_tab2 = Table.read('as1063_clump_properties.fits')
as1063_tab2['clump_nb'] = range(len(as1063_tab2))

a521_tab2 = Table.read('a521_clump_properties.fits')
a521_tab2['clump_nb'] = range(len(a521_tab2))

snake_tab2 = Table.read('snake_clump_properties.fits')
snake_tab2['clump_nb'] = range(len(snake_tab2))
#snake_tab2.remove_row(7)

Table

In [14]:
prepare_latex_table2(as1063_tab2,'AS-') 

head_a521 = a521_tab2[np.where(a521_tab2['mlt_region']==1)]
prepare_latex_table2(head_a521,'A-H-') 

tailN_a521 = a521_tab2[np.where(a521_tab2['mlt_region']==2)]
prepare_latex_table2(tailN_a521,'A-E-') 

tailS_a521 = a521_tab2[np.where(a521_tab2['mlt_region']==3)]
prepare_latex_table2(tailS_a521,'A-W-') 

north_snake = snake_tab2[np.where(snake_tab2['mlt_region']==1)]
prepare_latex_table2(north_snake,'M-N-') 

centre_snake = snake_tab2[np.where(snake_tab2['mlt_region']==2)]
prepare_latex_table2(centre_snake,'M-C-') 

south_snake = snake_tab2[np.where(snake_tab2['mlt_region']==3)]
prepare_latex_table2(south_snake,'M-S-')

AS-0 & 3.27$\pm$0.04 & 8881$^{+2754}_{-343}$ & 368$^{+205}_{-225}$ & 10.9$\pm$0.8 & 8.1$^{+0.3}_{-0.4}$ & 7.7$^{+0.7}_{-0.8}$ & 0.4 & 321$^{+207}_{-23}$ & 990$^{+295}_{-371}$ \\
AS-1 & 3.98$\pm$0.03 & 56637$^{+13749}_{-1212}$ & 1205$^{+148}_{-143}$ & 82.4$\pm$2.7 & 8.9$^{+0.2}_{-0.2}$ & 9.9$^{+0.2}_{-0.2}$ & 12.9 & 3362$^{+1176}_{-107}$ & 589$^{+137}_{-121}$ \\
AS-2 & 4.35$\pm$0.07 & 7405$^{+118}_{-150}$ & 424$^{+3}_{-135}$ & 41.2$\pm$0.4 & 8.0$^{+0.0}_{-0.2}$ & 8.9$^{+0.0}_{-0.3}$ & 8.7 & 2514$^{+95}_{-82}$ & 620$^{+0}_{-116}$ \\
AS-3 & 4.22$\pm$0.03 & 16023$^{+4938}_{-534}$ & 791$^{+164}_{-171}$ & 37.3$\pm$0.5 & 8.3$^{+0.3}_{-0.3}$ & 9.1$^{+0.2}_{-0.2}$ & 6.1 & 688$^{+278}_{-29}$ & 386$^{+116}_{-126}$ \\
AS-4 & 4.37$\pm$0.04 & 21710$^{+205}_{-216}$ & 853$^{+4}_{-43}$ & 20.4$\pm$0.1 & 8.5$^{+0.0}_{-0.1}$ & 8.6$^{+0.0}_{-0.1}$ & 1.5 & 222$^{+5}_{-2}$ & 451$^{+0}_{-40}$ \\
$\dagger$AS-5 & 4.48$\pm$0.05 & 3210$^{+878}_{-324}$ & 467$_{-470}$ & 12.0$\pm$9.8 & 7.6$^{+0.3}_{-1.0}$ & 7.8$^{+1